In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.neural_network import MLPRegressor, MLPClassifier

from sklearn.metrics import mean_squared_error, f1_score

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [3]:
# Инициализируем все известные генераторы случаынйх чисел / Setting all known random seeds
my_code = "Pyslar"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

tf.compat.v1.set_random_seed(my_seed)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [4]:
# Читаем данные из файла
example_data = pd.read_csv("../datasets/Fish.csv")

In [5]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [6]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [7]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [8]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']
ord_columns = ['Species']

ct = ColumnTransformer(transformers=[
    ('numerical', MinMaxScaler(), num_columns), 
    ('ordinal', OneHotEncoder(), ord_columns)])

ct.fit(train)

ColumnTransformer(transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width']),
                                ('ordinal', OneHotEncoder(), ['Species'])])

In [9]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [10]:
sc_train

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.000973,0.048544,0.038182,0.047297,0.000000,0.001659,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.063317,0.225243,0.225455,0.241554,0.251791,0.321337,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.391764,0.456311,0.456364,0.505068,0.750255,0.654770,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.057235,0.168932,0.174545,0.175676,0.195062,0.310741,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.069400,0.194175,0.192727,0.211149,0.385288,0.253046,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.178882,0.376699,0.369091,0.359797,0.336858,0.498599,0.0,0.0,1.0,0.0,0.0,0.0,0.0
91,0.020741,0.122330,0.120000,0.121622,0.111142,0.183539,0.0,0.0,1.0,0.0,0.0,0.0,0.0
92,0.430692,0.489320,0.501818,0.552365,0.872871,0.672486,1.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.665470,0.633010,0.629091,0.619932,0.632554,0.896262,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
# Устанавливаем названия столбцов
column_names = num_columns + list(range(7))
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [12]:
# В качестве входных параметров используем первые 5 числовых параметров,
# в качестве выходного - шестой числовой параметр.
x_labels = num_columns[:-1]
y_labels = num_columns[-1]
print(x_labels)
print(y_labels)

['Weight', 'Length1', 'Length2', 'Length3', 'Height']
Width


In [13]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]

In [14]:
# Создадим нейроннную сеть для решения задачи регрессии на базе библиотеки sklearn
reg = MLPRegressor(alpha=0.0, batch_size=16, epsilon=1e-07, max_iter=50)
reg.get_params()

{'activation': 'relu',
 'alpha': 0.0,
 'batch_size': 16,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-07,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 50,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [15]:
# Обучим нейронную сеть
reg.fit(x_train, y_train)

MLPRegressor(alpha=0.0, batch_size=16, epsilon=1e-07, max_iter=50)

In [16]:
# Проверим работу обученной нейронной сети на валидационной выборке
pred_val = reg.predict(x_val)
mse1 = mean_squared_error(y_val, pred_val)
print(mse1)

0.005623815747869495


In [17]:
# Создадим нейронную сеть на базе библиотеки keras
model = Sequential()

model.add(Dense(100, input_dim=5, activation='relu', use_bias=False))

model.add(Dense(1, activation='linear', use_bias=False))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               500       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 100       
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Обучим нейронную сеть
history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=50, batch_size=16, verbose=0)

In [19]:
# Проверим работу обученной нейронной сети на валидационной выборке
pred_val2 = model.predict(x_val)
mse2 = mean_squared_error(y_val, pred_val2)
print(mse2)

0.006969075183511124


In [20]:
# Создаем нейронную сеть со слоем дропаута
drop = Sequential()

drop.add(Dense(100, input_dim=5, activation='relu', use_bias=False))

drop.add(Dropout(rate=0.5))

drop.add(Dense(1, activation='linear', use_bias=False))

drop.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

drop.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               500       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 100       
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Обучим нейронную сеть
history = drop.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=50, batch_size=16, verbose=0)

In [22]:
# Проверим работу обученной нейронной сети на валидационной выборке
pred_val3 = drop.predict(x_val)
mse3 = mean_squared_error(y_val, pred_val3)
print(mse3)

0.009435034948166825


In [23]:
print(mse1, mse2, mse3)

0.005623815747869495 0.006969075183511124 0.009435034948166825


In [24]:
# Проверяем на тестовой выборке
pred_test = model.predict(x_test)
mse = mean_squared_error(y_test, pred_test)
print(mse)

0.004361568952645394


In [25]:
# Задание №1 - решение задачи классификации.
# В качестве входных параметров используем все числовые параметры,
# в качестве выходного - единственный категориальный параметр.

In [26]:
x_labels = num_columns
y_labels = list(range(7))
print(x_labels)
print(y_labels)

['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']
[0, 1, 2, 3, 4, 5, 6]


In [27]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]

In [28]:
# Создайте нейронную сеть для решения задачи классификации двумя способами: с помощью sklearn и keras.
# Сравните их эффективность.
# Для keras используйте loss и metrics = 'categorical_crossentropy'.

In [29]:
# sklearn
clas = MLPClassifier(alpha=0.0, batch_size=16, epsilon=1e-07, max_iter=50)
clas.get_params()
clas.fit(x_train, y_train)
pred_val = clas.predict(x_val)
mse1 = mean_squared_error(y_val, pred_val)
print(mse1)

0.13392857142857142


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [30]:
# keras
model = Sequential()

model.add(Dense(100, input_dim=6, activation='relu', use_bias=False))

model.add(Dense(7, activation='softmax', use_bias=False))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_crossentropy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               600       
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 700       
Total params: 1,300
Trainable params: 1,300
Non-trainable params: 0
_________________________________________________________________


In [31]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=16, verbose=0)

In [32]:
pred_val2 = model.predict(x_val)
mse2 = mean_squared_error(y_val, pred_val2)
print(mse2)

0.09273668756008745


In [33]:
# Задание №2 - использование dropout-слоя.

In [34]:
drop = Sequential()

drop.add(Dense(100, input_dim=6, activation='relu', use_bias=False))

drop.add(Dropout(rate=0.1))

drop.add(Dense(7, activation='softmax', use_bias=False))

drop.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_crossentropy'])

drop.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               600       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 700       
Total params: 1,300
Trainable params: 1,300
Non-trainable params: 0
_________________________________________________________________


In [35]:
history = drop.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=50, batch_size=16, verbose=0)

In [36]:
pred_val3 = drop.predict(x_val)
mse3 = mean_squared_error(y_val, pred_val3)
print(mse3)

0.09476725408317734


In [37]:
print(mse1, mse2, mse3)

0.13392857142857142 0.09273668756008745 0.09476725408317734


In [38]:
# Создайте нейронную сеть для решения задачи классификации с помощью keras. Используйте dropout-слой.
# Сравните эффективность нейронных сетей с dropout-слоем и без него.
# Попробуйте найти такие параметры dropout-слоя, чтобы сеть с ним работала лучше, чем без него.

In [39]:
# Проверяем на тестовой выборке
pred_test = model.predict(x_test)
mse = mean_squared_error(y_test, pred_test)
print(mse)

0.09348434513835933
